### Introduction to Nilearn and image manipulation

The goal of this notebook is to help get you comfortable with manipulating functional and anatomical images using nilearn. We'll be using the techniques we learned here in our final analysis...

#### Content:
1. Basic Image Operations and Masking
2. Resampling data to work across modalities (T1/FUNC)

In [ ]:
#Imports

In [ ]:
#Matplotlib magic

Let's grab some sample images from fmriprep

In [ ]:
#Base directory for fmriprep output
fmriprep_dir = '../data/ds000030/derivatives/fmriprep/{subject}/{mod}/'
t1_dir = fmriprep_dir.format(subject='sub-10788', mod='anat') 
func_dir = fmriprep_dir.format(subject='sub-10788', mod='func')

In [ ]:
#Display files inside of anatomy folder 


### Warming up with Nilearn 

#### Basic Image operations
Here we show how easy it is to apply a mask to nifti images using nilearn. 
We first pick an example T1 image and mask from the fmriprep output, then we apply the mask and view the result...

In [ ]:
ex_T1 = os.path.join(t1_dir,'sub-10788_T1w_preproc.nii.gz')
ex_bm = os.path.join(t1_dir,'sub-10788_T1w_brainmask.nii.gz')

In [ ]:
#Viewing T1


Nilearn uses nibabel under the hood to store nifti images. So you can think of images (similar to nibabel) as a 3D array (volume) with values assigned to each (x,y,z) coordinate. With 3D arrays we can perform operations and nilearn makes simple

Say we wanted to invert a T1 image, that is flip the colorscale. We could do this as follows:

In [ ]:
#Inverting a T1 image example


Now we could use a similar technique to apply a mask. 
A mask is simply a 3D volume with 0's and 1's. The 1's indicate the areas in which areas are to be preserved. 

In [ ]:
#Viewing mask


The idea behind applying a mask is multiplying the mask directly by the image in an element-wise fashion. 
The end result is that all voxels (x,y,z) in the T1 image corresponding with a 1 in the mask with the same coordinate (x,y,z) is preserved. Everything else is put to 0.

In [ ]:
#Applying a mask

**Exercise** 

Try applying the mask but getting everything but the brain!

In [ ]:
#Solution


#### Resampling
Recall from our nibabel exploration of neuroimaging data:

- T1 images are typically composed of voxels that are 1x1x1 in dimension
- Functional images are typically composed of voxels that are 2x2x2 in dimension

If we'd like to overlay our functional on top of our T1 (for visualization purposes, or analyses), then we need to match the size of the voxels! 

Think of this like trying to overlay a 10x10 JPEG and a 20x20 JPEG on top of each other. To get perfect overlay we need to resize (or more accurately *resample*) our JPEGs to match!

**Note**: 
Resampling is a method of interpolating in between data-points. When we stretch an image we need to figure out what goes in the spaces that are created via stretching - resampling does just that. In fact, resizing any type of image is actually just resampling to new dimensions. 

Let's resampling some MRI data using nilearn. 

**Goal**: Match the dimensions of the structural image to that of the functional image

In [ ]:
#Files we'll be using (Notice that we're using _space-MNI..._ which means they are normalized brains)
ex_mni_T1 = os.path.join(t1_dir,'sub-10788_T1w_space-MNI152NLin2009cAsym_preproc.nii.gz')
ex_mni_FUNC = os.path.join(func_dir,'sub-10788_task-rest_bold_space-MNI152NLin2009cAsym_preproc.nii.gz')

Note below that when we load an image, it reads in as a nibabel image. 
Meaning we can use the same utilities we used in nibabel on this data!

Resampling in nilearn is as easy as telling it which image you want to sample and what the target image is.
Structure of function:

<code>img.resample_to_img(source_img,target_img,interpolation)</code>
1. <code>source_img</code> - the image you want to sample
2. <code>target_img</code> - the image you wish to *resample to* 
3. <code>interpolation</code> - the method of interpolation

A note on **interpolation**

nilearn supports 3 types of interpolation, the one you'll use depends on the type of data you're resampling!
1. **continuous** - Interpolate but maintain some edge features.  Ideal for structural images where edges are well-defined. Uses $3^\text{rd}$-order spline interpolation.
2. **linear (default)** - Interpolate uses a combination of neighbouring voxels - will blur. Uses trilinear interpolation.
3. **nearest** - matches value of closest voxel (majority vote from neighbours). This is ideal for masks which are binary since it will preserve the 0's and 1's and will not produce in-between values (ex: 0.342). Also ideal for numeric labels where values are 0,1,2,3... (parcellations). Uses nearest-neighbours interpolation with majority vote.


In [ ]:
#Try playing around with methods of interpolation
#options: 'linear','continuous','nearest'

**Exercise**

Using **Native** T1 and **T1w** resting state functional do the following:
1. Resample the native T1 image to resting state size
2. Replace the brain in the T1 image with the first frame of the resting state brain

In [1]:
#Files we'll need

#This is the pre-processed T1 data that hasn't been standardized
ex_T1 = os.path.join(t1_dir,'sub-10171_T1w_preproc.nii.gz')

#This is the brain mask of the above T1
ex_t1_bm = os.path.join(t1_dir,'sub-10171_T1w_brainmask.nii.gz')

#This is the pre-processed resting state data that hasn't been standardized
ex_func = os.path.join(func_dir,'sub-10171_task-rest_bold_space-T1w_preproc.nii.gz')

#This is the associated mask for the resting state image.
ex_func_bm = os.path.join(func_dir,'sub-10171_task-rest_bold_space-T1w_brainmask.nii.gz')

NameError: name 'os' is not defined

The first step we need to do is to make sure the dimensions for our T1 image and resting state image match each other:

In [ ]:
#Resample the T1 to the size of the functional image!
resamp_t1 = img.resample_to_img(source_img=??, target_img=??, interpolation='continuous')
plot.plot_anat( ?? )


Next we want to make sure that the brain mask for the T1 is also the same dimensions as the functional image. This is exactly the same as above, except we use the brain mask as the source.

What kind of interpolation should we use for masks? 

In [ ]:
resamp_bm = ??
#Plot the image!



Once we've resampled both our T1 and our brain mask. We now want to remove the brain from the T1 image so that we can replace it with the funtional image instead. Remember to do this we need to:

1. Invert the T1 mask
2. Apply the inverted mask to the brain

In [ ]:
inverted_bm_t1 = img.math_img('??',a=??)

#Plot the image!

Now apply the mask:

In [5]:
resamp_t1_nobrain = ??

#Plot the image!

SyntaxError: invalid syntax (<ipython-input-5-eeeba05a1dea>, line 1)

We now have a skull missing the structural T1 brain. The final steps is to stick in the brain from the functional image into the now brainless head. First we need to remove the surrounding signal from the functional image.

Since a functional image is 4-Dimensional, we'll need to pull the first volume to work with. This is because the structural image is 3-dimensional and operations will fail if we try to mix 3D and 4D data.

In [ ]:
#Let's visualize the first volume of the functional image:
func_img = img.load_img(??)

#Pick the first volume (index 0) using "slicer"
first_vol = ??.slicer[?,?,?,?]

#Plot the first volume

As shown in the figure above, the image has some "signal" outside of the brain. In order to place this within the now brainless head we made earlier, we need to mask out the functional MR data as well!

In [ ]:
#Mask first_vol using ex_func_bm
masked_func = ??

#Plot the masked functional image

The final step is to stick this data into the head of the T1 data. Since the hole in the T1 data is represented as $0$'s. We can add the two images together to place the functional data into the void:

In [ ]:
#Add the two images together
combined_img = ??

#Plot!

### Finished!
Hopefully you've gained some intuition and comfort in dealing with MRI images using nilearn/nibabel. 

Now we'll take what we've used here, add some additional features and perform a full analysis on a dataset